In [ ]:
import torch
import torch.nn as nn
from datasets.plane import Plane
from torch.utils.data import DataLoader
import random
import numpy as np
from sklearn.metrics import classification_report, accuracy_score
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# set random seed
seed = 777
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
# test_loader
print('loading test set ...')
test_set = Plane(train=False)
test_load = DataLoader(dataset=test_set, batch_size=2, num_workers=4, shuffle=True)

In [ ]:
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
# set params; net: vgg, densenet        
model = 'vgg16' 
# select and load an model
if model == 'vgg19':
    from models.vgg19 import Net 
elif model == 'vgg16':
    from models.vgg16 import Net 
elif model == 'vgg13':
    from models.vgg13 import Net 
elif model == 'densenet':
    from models.densenet import Net
else:
    print('model: {} is not support!'.format(model))
net = Net()
net = nn.DataParallel(net, device_ids=[0])
net.load_state_dict(torch.load('outputs/best_{}_freeze.pth'.format(model))) 
net.cuda()
net.eval()


In [ ]:
# evaluation
def evaluate(net):
    net.eval()
    y_label = []
    y_preds = []
    with torch.no_grad():
        for sample in test_load:
            im, gt = sample
            im, gt = im.cuda(), gt.cuda()
            output = net(im)
            _, preds = torch.max(output, dim=1)
            y_preds.append(preds)
            y_label.append(gt)
    y_label = torch.cat(y_label, dim=0).detach().cpu().numpy()
    y_preds = torch.cat(y_preds, dim=0).detach().cpu().numpy()
    print(classification_report(y_label, y_preds))
    acc = accuracy_score(y_label, y_preds)
    print('Acc: {}'.format(acc))
evaluate(net)